In [ ]:
import cv2
import math
import numpy as np
import time
from collections import defaultdict
from ultralytics import YOLO

# Function: Perform object detection on a specific frame
def detect_objects(frame, model, class_names, track_history, next_id):
    results = model(frame, stream=True)

    for r in results:
        boxes = r.boxes

        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # Draw bounding box for the detected object
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 255), 3)

            # Calculate detection confidence
            confidence = math.ceil((box.conf[0] * 100)) / 100

            # Get the class name of the detected object
            class_id = int(box.cls[0])
            class_name = class_names[class_id]

            # Check if this box is already being tracked
            track_id = None
            for tid, history in track_history[class_name].items():
                if len(history) > 0:
                    last_position = history[-1]
                    dist = np.linalg.norm(np.array(last_position) - np.array([(x1 + x2) // 2, (y1 + y2) // 2]))
                    if dist < 50:  # Adjust this threshold as needed
                        track_id = tid
                        break

            # If not being tracked, assign a new track ID
            if track_id is None:
                track_id = next_id[class_name]
                next_id[class_name] += 1

            # Display the class name and confidence of the detected object on the frame
            text = f"{class_name} {track_id}: {confidence:.2f}"
            org = (x1, y1 - 10)
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.5
            color = (255, 0, 0)
            thickness = 1

            cv2.putText(frame, text, org, font, font_scale, color, thickness)

            # Store tracking history
            track = track_history[class_name][track_id]
            track.append(((x1 + x2) // 2, (y1 + y2) // 2))
            if len(track) > 30:
                track.pop(0)

            # Plot tracks
            points = np.array(track, dtype=np.int32).reshape((-1, 1, 2))
            cv2.circle(frame, track[-1], 7, (0, 255, 0), -1)
            cv2.polylines(frame, [points], isClosed=False, color=(0, 255, 0), thickness=2)

# Function: Read frames from the camera and perform object detection
def capture_frames(camera, window_name, model, class_names, track_history, next_id, target_height=480):
    start_time = time.time()
    frame_count = 0

    while True:
        success, frame = camera.read()

        if success:
            # Resize frame to have a consistent height
            height, width, _ = frame.shape
            target_width = int(target_height * (width / height))
            frame = cv2.resize(frame, (target_width, target_height))

            # Perform object detection on the frame
            detect_objects(frame, model, class_names, track_history, next_id)

            # Calculate and display FPS
            frame_count += 1
            elapsed_time = time.time() - start_time
            fps = frame_count / elapsed_time
            cv2.putText(frame, f"FPS: {fps:.2f}", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Display the frame in the window
            cv2.imshow(window_name, frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

# Start the PC Webcam
camera = cv2.VideoCapture(0)
camera.set(3, 1280)
camera.set(4, 720)

# Start the YOLO model
yolo_model = YOLO("bestg_544.pt")

# Object classes
class_names = ["person"]

# Tracking history and ID management
track_history = defaultdict(lambda: defaultdict(list))
next_id = defaultdict(lambda: 1)

# Start capturing frames and performing object detection
capture_frames(camera, 'PC Camera', yolo_model, class_names, track_history, next_id)

# Release the camera and close OpenCV windows
camera.release()
cv2.destroyAllWindows()


